# MATH50003 Numerical Analysis (2024–2025) Computer-based Exam

Instructions:

1. You have 15 mins to read the exam beginning when the invigilators instruct. **DO NOT** write or type anything during this time.
2. You have 1 hour to complete the exam beginning when the invigilators instruct. You **MUST STOP** typing when the time is complete.
3. When finished, save your work and close Visual Studio Code.
4. Re-access WISEflow in SchoolYear and click on ‘Upload Paper’ → From Computer.
6. You can now complete submission by clicking the green button "Click here to submit".
6. If uploading fails please contact an invigilator.

Instructions for the exam:

1. For each problem, replace the `# TODO` to complete the question.
The unit tests are provided to help you test your answers, but do not guarantee that the answer is correct.
2. Problems are marked A/B/C to indicate difficulty ("A" being most difficult).
3. All questions are worth 10 marks. Partial credit will be awarded for reasonable attempts or comments outlining a solution even if the tests
are not passed.
3. If you have technical queries please contact an invigilator.
4. You may use existing code from the module Github page.
5. You **MUST NOT** ask for help online or
communicate with others within or outside the module.
Failure to follow these rules will be considered academic misconduct.
6. **NO USAGE of AI tools** such as ChatGPT or GitHub Co-Pilot.

You should use the following packages:

In [12]:
using LinearAlgebra, SetRounding, Test

**WARNING** It may be necessary to restart the kernel if issues arise. Remember to reload the packages
when you do so.

-----

**Problem 1 (B)** The error in the right rectangular rule behaves like
$$
∫_0^1 f(x) {\rm d}x = {1 \over n} ∑_{j=1}^n f(x_j) + B_f/n + O(n^{-2})
$$
for some unknown constant $B_f$. Find a linear combination of `rightrectangularrule(f,n)` and `rightrectangularrule(f,2n)`
that cancels out the $B_f/n$ terms and thereby returns an approximation to an integral that has error that behaves like $O(n^{-2})$.

In [13]:
function rightrectangularrule(f, n)
    ret = 0.0
    for j = 1:n
        ret = ret + f(j/n)
    end
    ret/n
end

function rombergrule(f, n)
    # TODO: combine rightrectangularrule(f, n) and rightrectangularrule(f, 2n) to get a high accuracy approximation to the integral
    # SOLUTION
    # 6 points for derivation
    # Consider ∫_0^1 f(x) {\rm d}x = (2-1) ∫_0^1 f(x) {\rm d}x =
    #  2*{1 \over 2n} ∑_{j=1}^n f(x_j) + B_f/(n) - {1 \over n} ∑_{j=1}^n f(x_j) - B_f/n + O(n^2)
    # 2 points for calling rightrectangularule
    2rightrectangularrule(f,2n)-rightrectangularrule(f,n)
    # END
end

@test rombergrule(exp,10_000)  ≈ exp(1)-1 # achieves higher accuracy than rightrectangularrule on its own

Test Passed

**Problem 2 (C)** Use central differences with a careful choice of `h` to approximate the derivative of $g(x) = ∏_{k=1}^{100} \left({x \over 2k}-1\right)$
with an absolute accuracy of $10^{-10}$.

In [14]:
function g(x)
    # TODO: implement the specified function g
    # SOLUTION
    # 2 points for 4 loop, 3 more points for correct implementation
    ret = 1.0
    for k = 1:100
        ret = ret * (x /(2k) - 1)
    end
    ret
    # END
end


function centraldifferenceswithg()
    # TODO: apply central differences for the prescribed function g to approximate its derivative at 0.1
    # SOLUTION
    # 3 points for correct central differences, 2 points for choosing h so that test passes
    h = cbrt(eps()) # use the heuristic
    (g(0.1+h) - g(0.1-h))/(2h)
    # END
end

@test centraldifferenceswithg() ≈ -2.029623096202721 atol = 1E-10

Test Passed

**Problem 3 (B)** Implement `*` and `+` for a 2D analogue of dual numbers, representing
$$
a + bϵ_x + cϵ_y
$$
such that $ϵ_xϵ_y = ϵ_x^2 = ϵ_y^2 = 0$.

In [15]:
struct Dual2D
    a
    b
    c
end

import Base: +, *

function +(x::Dual2D, y::Dual2D)
    # TODO: return a Dual2D corresponding to the addition of x and y
    # SOLUTION
    # 4 points for correct definition
    Dual2D(x.a+y.a, x.b+y.b, x.c+y.c)
    # END
end

function *(x::Dual2D, y::Dual2D)
    # TODO: return a Dual2D corresponding to the multiplication of x and y
    # SOLUTION
    # 6 points for correct definition
    Dual2D(x.a*y.a, x.a*y.b+x.b*y.a, x.a*y.c+x.c*y.a)
    # END
end

@test Dual2D(1,2,3) + Dual2D(4,5,6) == Dual2D(5,7,9)
@test Dual2D(1,2,3) * Dual2D(4,5,6) == Dual2D(4, 13, 18)

Test Passed

**Problem 4 (C)**  Implement `haslargestexponent(x::Float16)` that returns true if `x` is a positive normal
half-precision float ($F_{16} = F_{15,5,10}$)  which has the largest possible exponent.

In [16]:
function haslargestexponent(x::Float16)
    # TODO: deduce if x is positive, normal and has the exponent that is the largest possible
    # SOLUTION
    if x ≤ 0
        return false # 2 points for special case
    end
    # 4 points for using bitstring
    # 2 points for taking a substring
    bitstring(x)[2:6] == "11110"
    # END
end

@test !haslargestexponent(Float16(Inf))
@test !haslargestexponent(Float16(3.275e4))
@test haslargestexponent(Float16(3.277e4))
@test haslargestexponent(Float16(3.28e4))
@test !haslargestexponent(Float16(-3.28e4))

Test Passed

**Problem 5.1 (C)**  Implement `intersect` and  `union` for intervals. You may assume the
intervals overlap.

In [17]:
import Base: intersect, union, in

struct Interval # represents the set [a,b]
    a # left endpoint
    b # right endpoint
end

Interval(x) = Interval(x,x) # Support Interval(1) to represent [1,1]

in(x, X::Interval) = X.a ≤ x ≤ X.b

function intersect(X::Interval, Y::Interval)
    a,b = X.a,X.b
    c,d = Y.a,Y.b
    if b < c || d < a
        error("Intervals must overlap")
    end
    # TODO: Create an interval corresponding to the intersection of X = [a,b] and Y = [c,d]
    # SOLUTION
    # 2 points for using max and min, 3 points for correct result
    Interval(max(a,c),min(b,d))
    # END
end

function union(X::Interval, Y::Interval)
    a,b = X.a,X.b
    c,d = Y.a,Y.b
    if b < c || d < a
        error("Intervals must overlap")
    end
    # TODO: Create an interval corresponding to the union of X = [a,b] and Y = [c,d]
    # SOLUTION
    # 2 points for using max and min, 3 points for correct result
    Interval(min(a,c),max(b,d))
    # END
end

@test intersect(Interval(1,3), Interval(2,4)) == intersect(Interval(2,4), Interval(1,3)) == Interval(2,3)
@test union(Interval(1,3), Interval(2,4)) == union(Interval(2,4), Interval(1,3)) == Interval(1,4)

Test Passed

**Problem 5.2 (A)** Interval Newton's method combines Newton's method with interval arithemtic via the iteration
$$
X_{k+1} = \left(m_k - {f(m_k) \over f'(X_k)}\right) ∩ X_k
$$
where `m_k` is an arbitrary point in `X_k`, for example, the midpoint.
Implement `\(c::Number, X::Interval)` and
this iteration in `intervalnewton(f, fp, X_0, n)` and thereby compute an interval $X_n$ containing the
root of `f`, given its derivative `fp` and an initial guess `X_0`.

In [18]:
import Base: +, -, *, /, ^
function +(X::Interval, Y::Interval)
    a,b,c,d = promote(X.a, X.b, Y.a, Y.b) # make sure all are the same type
    T = typeof(a)
    α = setrounding(T, RoundDown) do
        a + c
    end
    β = setrounding(T, RoundUp) do
        b + d
    end
    Interval(α, β)
end

function -(X::Interval, Y::Interval)
    a,b,c,d = promote(X.a, X.b, Y.a, Y.b)
    T = typeof(a)
    α = setrounding(T, RoundDown) do
        a - d
    end
    β = setrounding(T, RoundUp) do
        b - c
    end
    Interval(α, β)
end

+(x::Number, Y::Interval) = Interval(x) + Y # Number is a supertype that contains Int, Float64, etc.
-(x::Number, Y::Interval) = Interval(x) - Y # Number is a supertype that contains Int, Float64, etc.
+(X::Interval, y::Number) = X + Interval(y)
-(X::Interval, y::Number) = X - Interval(y)

function *(X::Interval, Y::Interval)
    a,b,c,d = promote(X.a, X.b, Y.a, Y.b)
    T = typeof(a)
    if !(0 < a ≤ b && 0 < c ≤ d)
        error("Input doesn't satisfy positivity assumptions")
    end
    α = setrounding(T, RoundDown) do
            a * c
    end
    β = setrounding(T, RoundUp) do
            b * d
    end
    Interval(α, β)
end

*(c::Number, X::Interval) = Interval(c) * X

function ^(X::Interval, k::Int)
    if k ≤ 0
        error("not supported")
    elseif k == 1
        X
    else
        X * X^(k-1)
    end
end

function /(c::Number, X::Interval)
    a,b = X.a,X.b
    if !(0 < a < b)
        error("Only positive intervals are supported")
    end
    # TODO: implement division with correct rounding.  You may assume a, b, and c are all Float64.
    # a and b are positive but c may be negative.
    # SOLUTION
    # 2 points for using setrounding, 2 points for correct result
    α = setrounding(Float64, RoundDown) do
        if c > 0
            c / b
        else
            c / a
        end
    end
    β = setrounding(Float64, RoundUp) do
        if c > 0
            c / a
        else
            c / b
        end
    end
    Interval(α, β)
    # END
end

function intervalnewton(f, fp, X_0::Interval, n)
    # TODO: implement interval Newton's method. f(m_k) can be computed with standard floating point.
    # SOLUTION
    # 2 points for a Newton-like 4 loop
    X_k = X_0
    for k = 1:n
        m_k = (X_k.b + X_k.a)/2 # 2 points for creating m_k inside X_k
        X_k = intersect(m_k - f(m_k) / fp(X_k), X_k) #2 points for correct iteration, full marks if the - is a + due to typo in exam
    end
    X_k
    # END
end

@test 1.0 / Interval(3,5) == Interval(0.19999999999999998, 0.33333333333333337)
@test (-1.0) / Interval(3,5) == Interval(-0.33333333333333337, -0.19999999999999998)

f = x -> x^3 - 3x + 1
fp = x -> 3x^2 - 3
X = intervalnewton(f, fp, Interval(1.5,1.7), 6)
@test X.b - X.a ≤ 10^(-7)
@test 1.5320888862379562 in X

Test Passed

**Problem 6 (A)** Implement `reversecholesky(A::SymTridiagonal)` that returns an upper-bidiagonal matrix `U` such that `U*U' ≈ A`,
for the special case where `A` is symmetric tridiagonal, using only $O(n)$ operations.
You may assume the input is symmetric positive definite and has `Float64` values. You must not use the inbuilt `cholesky`
function or in any other way reduce the problem to a standard Cholesky decomposition.

In [19]:
function reversecholesky(A::SymTridiagonal)
    n = size(A,1)
    U = Bidiagonal(zeros(n), zeros(n-1), :U)
    A = copy(A) # you may wish to alter a copy of A in-place
    # TODO: populate U so that U*U' ≈ A
    # SOLUTION
    for j = n:-1:2 # 3 points for recognising it needs to start at bottom right
        α,v = A[j,j],A[j-1,j]
        if α ≤ 0
            error("Matrix is not SPD") # this error is optional
        end
        U[j,j] = sqrt(α)
        U[j-1,j] = v/sqrt(α) # 4 points if this entry is chosen

        # update, 2 points if correct
        A[j-1,j-1] = A[j-1,j-1] - v^2/α
    end
    U[1,1] = sqrt(A[1,1])
    # END
    U
end

A = SymTridiagonal([2.0,2,2], [1.0,1])
U = reversecholesky(A)
@test U*U' ≈ A

Test Passed

**Problem 7 (A)**  Complete the definition of multiplication by a `BidiagonalReflections` which supports a sequence of reflections,
that is,
$$
Q = Q_{𝐯_1} ⋯ Q_{𝐯_n}
$$
where the vectors are stored as a lower bidiagonal matrix $V ∈ ℝ^{n × n}$ whose $j$-th column is $𝐯_j∈ ℝ^n$, and
$$
Q_{𝐯_j} = I - 2 𝐯_j 𝐯_j^⊤
$$
is a reflection. Ensure multiplication uses only $O(n)$ operations.

In [20]:
struct BidiagonalReflections <: AbstractMatrix{Float64}
    V::Bidiagonal{Float64} # Columns of V are the householder vectors
end

import Base: size, *
size(Q::BidiagonalReflections) = (size(Q.V,1), size(Q.V,1))


function *(Q::BidiagonalReflections, x::AbstractVector)
    m,n = size(Q)
    if n ≠ length(x)
        error("the dimensions must match")
    end
    # TODO: Apply Q in O(n) operations assuming that Q.V is a lower bidiagonal matrix.
    # SOLUTION
    x = copy(x) # I will do this in-place
    x[n] = x[n] - 2Q.V[n,n]^2*x[n] ## 2 points for n = 1 special case
    for j = n-1:-1:1 # 2 points for correct 4 loop
        # Householder reflection is (I - 2v*v')*x = x - 2v * (v'x) but we need to do this in O(1) operations
        μ = Q.V[j,j]*x[j] + Q.V[j+1,j]*x[j+1] # 2 points for computing the dot product in O(1) opeations
        x[j+1] = x[j+1] - 2Q.V[j+1,j]*μ # 4 points for correctly updating x[j+1] and x[j]
        x[j] = x[j] - 2Q.V[j,j]*μ
    end
    x
    # END
end

V = Bidiagonal([1/sqrt(2), 2/sqrt(5), 3/sqrt(10), 1], [1/sqrt(2), 1/sqrt(5), 1/sqrt(10)], :L)
Q = BidiagonalReflections(V);
x = [1.0,2,3,4]
@test Q * x ≈ (I-2V[:,1]V[:,1]')*(I-2V[:,2]V[:,2]')*(I-2V[:,3]V[:,3]')*(I-2V[:,4]V[:,4]')*x

Test Passed

**Problem 8 (C)** Approximate $\cos x$ by a quartic polynomial by minimising
the least squares error when sampled at $n$ evenly spaced points in $[0,1]$,
that is, $x_k = (k-1)/(n-1)$,
returning the coefficients in the monomial basis.

In [21]:
function cosfit(n)
    # TODO: return the coefficients [c_0,c_1,c_2,c_3,c_4] of the polynomial
    # c_0 + c_1*x + c_2*x^2 + c_3*x^3 + c_4*x^4 that minimises the 2-norm error
    # of approximating cos(x) at n evenly spaced samples
    # SOLUTION
    x = range(0,1; length=n) # 3 points for correct grid
    V = x .^ (0:4)' # 3 points for correct Vandermonde matrix
    V \ cos.(x) # 4 points for correct resul
    # END
end

c₀,c₁,c₂,c₃,c₄ = cosfit(1000)
x = 0.1
@test abs(c₀ + c₁*x + c₂*x^2 + c₃*x^3 + c₄*x^4 - cos(x)) ≤ 1E-3

Test Passed

**Problem 9 (B)** Consider a finite difference method for solving
$$
\begin{align*}
u(0) &= 1, u'(t) - \cos(t) u(t) = t
\end{align*}
$$
on the interval $[0,1]$. Construct a lower bidiagonal matrix $L ∈ ℝ^{n+1 × n+1}$ and vector $𝐛 ∈ ℝ^{n+1}$ so that
solving $L^{-1} 𝐛$ gives a vector $[u_0,…,u_n]^⊤$ so that $u_j ≈ u(x_j)$ for
$x_j = jh$ where $h = 1/n$, by
imposing the equation on the midpoints $x̃_1,…,x̃_n$ defined as
$$
x̃_j = {x_{j+1} + x_j \over 2} = (j-1/2)h
$$
using the central difference formula
$$
u'(x̃_j) ≈ {u_j - u_{j-1} \over h}
$$
alongside the approximation
$$
u(x̃_j) ≈ {u_j + u_{j-1} \over 2}.
$$

In [22]:
function midpointfinitedifference(n)
    # TODO: construct and return a tuple containing a lower bidiagonal matrix L and a vector b
    # corresponding to using central differences on the midpoints
    # SOLUTION
    ## 3 points for derivation in comment
    # Step 1: we write
    # [u(0),u'(x̃_1) - cos(x̃_1)u(x̃_1),…,u'(x̃_n) - cos(x̃_n)u(x̃_n)] = [1,x̃_1,…,x̃_n]
    # Step 2/3: replace with finite difference approximation
    # [u(0),(u_1-u_0)/h - cos(x̃_1)(u_1+u_0)/2,…,(u_n-u_{n-1})/h - cos(x̃_n)(u_n+u_{n-1})/2] = [1,x̃_1,…,x̃_n]
    # This is equivalent to the following linear system

    x = range(0, 1; length=n+1)
    h = step(x)
    x̃ = (x[2:end] + x[1:end-1])/2 ## 2 points for correctly construct midpoints
    L = Bidiagonal([1; 1/h .- cos.(x̃)/2], -1/h .- cos.(x̃)/2, :L) # 3 points for correct matrix
    L,[1;x̃] ## 2 points for correct RHS
    # END
end

n = 1_000
L,b = midpointfinitedifference(n)
@test L isa Bidiagonal
@test size(L) == (n+1,n+1)
@test L[n+1,n+1] ≈ n - cos(0.9995)/2
@test (L\b)[end] ≈ 2.967178119971284 atol=1E-5

Test Passed

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*